In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

c:\Users\Administrator\Desktop\FinetuningOptuna\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("heart.csv")
df = pd.get_dummies(df)
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)


In [8]:
model = RidgeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8804347826086957

In [14]:
def objective(trial):
    hiper = {
        "alpha" : trial.suggest_float("alpha", 0,100),
        #"fit_intercept": trial.suggest_categorical("fit_intercept", ["True", "False"]),
        #"copy_X": trial.suggest_categorical("copy_X", ["True", "False"]),
        "solver": trial.suggest_categorical("solver", ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])
    }
    model = RidgeClassifier(**hiper)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc  = accuracy_score(y_test, y_pred)
    return acc
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_jobs=-1, n_trials=1000)

[I 2025-08-22 20:03:03,560] A new study created in memory with name: no-name-fbb06955-cced-4c7c-a923-be5c48938e9f
[I 2025-08-22 20:03:03,596] Trial 2 finished with value: 0.894927536231884 and parameters: {'alpha': 47.572890770024316, 'solver': 'auto'}. Best is trial 2 with value: 0.894927536231884.
[I 2025-08-22 20:03:03,629] Trial 6 finished with value: 0.8985507246376812 and parameters: {'alpha': 55.793546831478125, 'solver': 'cholesky'}. Best is trial 6 with value: 0.8985507246376812.
[I 2025-08-22 20:03:03,639] Trial 0 finished with value: 0.8913043478260869 and parameters: {'alpha': 80.28330252517021, 'solver': 'lsqr'}. Best is trial 6 with value: 0.8985507246376812.
[I 2025-08-22 20:03:03,656] Trial 7 finished with value: 0.8985507246376812 and parameters: {'alpha': 50.04350076269915, 'solver': 'svd'}. Best is trial 6 with value: 0.8985507246376812.
[I 2025-08-22 20:03:03,658] Trial 9 finished with value: 0.8913043478260869 and parameters: {'alpha': 86.50094401572677, 'solver': 

In [18]:
model = RidgeClassifier(**study.best_params)
print(study.best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc  = accuracy_score(y_test, y_pred)
print(acc)

{'alpha': 55.793546831478125, 'solver': 'cholesky'}
0.8985507246376812
